 [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/SeoulTechPSE/EngNm/blob/master/ch10_code.ipynb)

# Chapter 10: Sparse matrices and graphs

Creator: Robert Johansson, Updator: Kee-Youn Yoo

Updated source code listings for Numerical Python - A Practical Techniques Approach for Industry 
<p>(ISBN 978-1-484205-54-9).

For matrices that are dominated by zero elements, it is inefficient to store all the zeros in the computer's memory, and it is more suitable to store only the nonzero values with additional information about their locations. For non-sparse matrices, known as dense matrices, such a representation is less efficient than storing all values consecutively in the memory, but for large *sparse* matrices it can be vastly superior

There are several options for working with sparse matrices in Python. Here we mainly focus on the sparse
matrix module in Scipy, [`scipy.sparse`](https://docs.scipy.org/doc/scipy/reference/sparse.html), which provides an easy-to-use interface for representing sparse matrices and carrying out linear algebra operations on such objects. Toward the end of the chapter, we also briefly explore representing and processing graphs, using the Scipy [`sparse.csgraph`](https://docs.scipy.org/doc/scipy/reference/sparse.csgraph.html) module and [the NetworkX library](https://networkx.github.io/). Graphs can be represented as adjacency matrices, which in many applications are very sparse. Graphs and sparse matrices are therefore closely connected topics

## Importing modules

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

import numpy as np
import scipy.linalg as la

import scipy.sparse as sp
import scipy.sparse.linalg

import networkx as nx

## Sparse matrices in scipy

The basic idea of sparse matrix representation is to avoid storing the excessive amount of zeros in a
sparse matrix. If we store only the nonzero elements, we clearly also need to store the row and column indices for each element. There are numerous approaches to organizing the storage of the nonzero elements and their corresponding row and column indices

The class [`sp.coo_matrix`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html) is used to represent sparse matrices in the coordinate list format. This format is particularly easy to initialize. For instance, with the matrix,

\begin{bmatrix}
0 & 1 & 0 & 0\\ 
0 & 0 & 0 & 2\\ 
0 & 0 & 3 & 0\\ 
4 & 0 & 0 & 0
\end{bmatrix}

we can easily identify the nonzero values $[A_{01}=1,\,A_{13}=2,\,A_{22}=3,\,A_{30}=4]$ and their corresponding rows $[0, 1, 2, 3]$ and columns $[1, 3, 2, 0]$ (note that here we have used Python’s zero-based indexing)

In [ ]:
values = [1, 2, 3, 4]

rows = [0, 1, 2, 3]
cols = [1, 3, 2, 0]

A = sp.coo_matrix((values, (rows, cols)), shape=[4, 4]); A

The result is a data structure that represents the sparse matrix. All sparse matrix representations in
Scipy's `sparse` module share several common attributes. Examples of such attributes are `shape`, `size`,`dtype`, and `ndim`, and common to all sparse matrix representations are the `nnz` (number of nonzero elements) and `data` (the nonzero values) attributes:

In [ ]:
A.shape, A.size, A.dtype, A.ndim

In [ ]:
A.nnz, A.data

In [ ]:
A.row, A.col

There are also a large number of methods available for operating on sparse matrix objects. Many of
these methods are for applying mathematical functions on the matrix. For example, element-wise math
methods like `sin`, `cos`, `arcsin`, etc., aggregation methods like `min`, `max`, `sum`, etc., mathematical array operators such as conjugate (`conj`) and transpose (`transpose`), etc., and `dot` for computing the dot product between sparse matrices or a sparse matrix and a dense vector

Another important family of methods is used to convert sparse matrices
between different formats: For example `tocoo`, `tocsr`, `tolil`, etc. There are also the methods for converting a sparse matrix to Numpy `ndarray` and Numpy `matrix` objects (that is, dense matrix representations): `toarray` and `todense`, respectively

In [ ]:
A.tocsr()

In [ ]:
A.toarray()

In [ ]:
A.todense()

The obvious way to access elements in a matrix, which we have used in numerous different contexts so
far, is using the indexing syntax, for example `A[1,2]`, as well as the slicing syntax, for example `A[1:3,2]`, and so on. We can often use this syntax with sparse matrices too, but not all representations support indexing and slicing, and if it is supported, it may not be an efficient operation

In [ ]:
A[0,1]

In [ ]:
A.tobsr()[0,1]

In [ ]:
A.tocsr()[0,1]

In [ ]:
A.tolil()[1:3,3]

When working with sparse matrices, it is common to face the situation that different tasks – such
as construction, updating, and arithmetic operations – are most efficiently handled in different formats.
Converting between different sparse formats is relatively efficient, so it is useful to switch between
different formats in different parts of an application. Efficient use of sparse matrices therefore requires an understanding of how different formats are implemented and what they are suitable for 

For computations, the most important sparse matrix representations in Scipy's `sparse` module are
the **CSR (Compressed Sparse Row)** and **CSC (Compressed Sparse Column)** formats, because they are well suited for efficient matrix arithmetic and linear algebra applications. Other formats, like **COO**, **LIL** and **DOK** are mainly used for constructing and updating sparse matrices, and once a sparse matrix is ready to be used in computations, it is best to convert it to either CSR or CSC format, using the `tocsr` or `tocsc` methods, respectively

In the CSR format, the nonzero values (`data`) are stored along with an array that contains the column
indices of each value (`indices`), and another array that stores the offsets of the column index array of each
row with the total number of nonzero elements at the end(`indptr`). For instance, consider the matrix

\begin{bmatrix}
1 & 3 & 0 & 0\\ 
0 & 5 & 7 & 0\\ 
0 & 0 & 9 & 11\\ 
13 & 0 & 15 & 17
\end{bmatrix}

In [ ]:
A = np.array([[1, 3, 0, 0], [0, 5, 7, 0], [0, 0, 9, 11], [13, 0, 15, 17]]); A

In [ ]:
A = sp.csr_matrix(A)

In [ ]:
A.data

In [ ]:
A.indices

In [ ]:
A.indptr

For example, the elements in the third row, with index `i = 2`, starts at
`indptr[2] = 4` and ends at `indptr[3] -1 = 5`, which gives the element values `data[4] = 9` and `data[5] = 11` and column indices `indices[4] = 2` and `indices[5] = 3`. Thus, `A[2,2] = 9` and `A[2,3] = 11`:

In [ ]:
i = 2
A.indptr[i], A.indptr[i +1] -1

In [ ]:
A.indices[A.indptr[i]:A.indptr[i +1]]

In [ ]:
A.data[A.indptr[i]:A.indptr[i +1]]

In [ ]:
A[2, 2], A[2, 3] # check

### Functions for creating sparse matrices

As we have seen examples earlier in this chapter, one way of constructing sparse matrices is to prepare the data structures for a specific sparse matrix format, and pass these to the constructor of the corresponding sparse matrix class. While this method is sometimes suitable, it is often more convenient to compose sparse matrices from predefined template matrices

To create a sparse matrix of
size 10 x 10 with a main diagonal and an upper and lower diagonal, we can use three calls to [`sp.eye`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.eye.html), using the `k` argument to specify the offset from the main diagonal:

In [ ]:
N = 10
A = sp.eye(N, k=1) -2 *sp.eye(N) +sp.eye(N, k=-1); A

By default, the resulting object is sparse matrix in the CSR format, but using the `format` argument, we
can specify any other sparse matrix format. All functions for creating sparse matrices in [`sp.sparse`](https://docs.scipy.org/doc/scipy/reference/sparse.html) accept this argument. For example, in the previous example, we could have produced the same matrix using [`sp.diags`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.spdiags.html), by specifying the pattern `[1, -2, 1]`, and the corresponding offsets from the main diagonal `[1, 0, -1]`. If we additionally want the resulting sparse
matrix in CSC format, we can set `format='csc'`: 

In [ ]:
A = sp.diags([1, -2, 1], [1, 0, -1], shape=[N, N], format='csc'); A

In [ ]:
fig, ax = plt.subplots()
ax.spy(A);

For example, to create a sparse matrix for the tensor product between $A$ and the matrix

$$\scriptsize
\begin{bmatrix}
0 & 1 & 0\\ 
1 & 0 & 1\\ 
0 & 1 & 0
\end{bmatrix}$$

we can use [`sp.kron`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.kron.html)`(A, B)`:

In [ ]:
B = sp.diags([1, 1], [-1, 1], shape=[3, 3])
C = sp.kron(A, B)

fig, (ax_A, ax_B, ax_C) = plt.subplots(1, 3, figsize=(12, 4))
ax_A.spy(A)
ax_B.spy(B)
ax_C.spy(C);

### Sparse linear algebra functions

The main application of sparse matrices is to perform linear algebra operations on large matrices that are intractable or inefficient to treat using dense matrix representations 

The Scipy `sparse` module contains a module `linalg` that implements many linear algebra routines. Not all linear algebra operations are suitable for sparse matrices, and in some cases the behavior of the sparse matrix version of operations needs to be modified compared to the dense counterparts

Consequently, there are a number of differences between the sparse linear algebra module [`scipy.sparse.linalg`](https://docs.scipy.org/doc/scipy/reference/sparse.linalg.html) and the dense linear algebra module [`scipy.linalg`](https://docs.scipy.org/doc/scipy/reference/linalg.html).
In general, for sparse matrix methods to be efficient, they must retain the sparsity of matrices involved in the computation

An examples of operations where the sparsity usually is not retained is the matrix inverse, and
it should therefore be avoided when possible

### Linear equation systems

The most important application of sparse matrices is arguably to solve linear equation system on the form
$\mathbf{A}\mathbf{x} = \mathbf{b}$, where $\mathbf{A}$ is a sparse matrix and $\mathbf{x}$ and $\mathbf{b}$ are dense vectors. 
The Scipy `sparse.linalg` module has both direct and iterative solver for this type of problem ([`sp.linalg.spsolve`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.spsolve.html)), and methods to factor a matrix $\mathbf{A}$, using for example LU factorization ([`sp.linalg.splu`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.splu.html)) and incomplete LU factorization ([`sp.linalg.spilu`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.spilu.html)).

In [ ]:
N = 10
A = sp.diags([1, -2, 1], [1, 0, -1], shape=[N, N], format='csc')
b = -np.ones(N)

In [ ]:
x = sp.linalg.spsolve(A, b); x

For comparison, we can also solve this problem using
dense direct solver in Numpy [`np.linalg.solve`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.solve.html) (or, similarly, using [`scipy.linalg.solve`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.solve.html))

In [ ]:
np.linalg.solve(A.todense(), b)

As expected, the result agrees with what we obtained from the sparse solver. For small problems like this
one there is not much to gain using sparse matrices, but for increasing system size the merits of using sparse
matrices and sparse solvers become apparent. For this particular problem, the threshold system size beyond which sparse methods outperforms dense methods is approximately `N=100`

An alternative to the [`spsolve`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.spsolve.html) interface is to explicitly compute the LU factorization using [`sp.sparse.splu`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.splu.html) or [`sp.sparse.spilu`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.spilu.html) (incomplete LU factorization). These functions return an object that contains the $\mathbf{L}$ and $\mathbf{U}$ factors, and that has a method `solve` that solves $\mathbf{LUx} = \mathbf{b}$ for a given vector $\mathbf{b}$

In [ ]:
lu = sp.linalg.splu(A)

In [ ]:
lu.L

In [ ]:
lu.U

In [ ]:
x = lu.solve(b); x

An important consideration that arises with sparse matrices is that the LU factorization of $\mathbf{A}$ may introduce new nonzero elements in $\mathbf{L}$ and $\mathbf{U}$ compared to the matrix $\mathbf{A}$, and therefore make $\mathbf{L}$ and $\mathbf{U}$ less sparse. Elements that exist in $\mathbf{L}$ or $\mathbf{U}$, but not in $\mathbf{A}$, are called fill-ins. 

If the amount of fill-ins is large, the
advantage of using sparse matrices may be lost. While there is no complete solution to eliminate fill-ins, it is often possible to reduce fill-in by permuting the rows and columns in $\mathbf{A}$, so that the LU factorization takes the form $\mathbf{P_r AP_c} = \mathbf{LU}$.

The [`spsolve`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.spsolve.html), [`splu`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.splu.html) and [`spilu`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.spilu.html) functions all take the argument `permc_spec`, which can take the values `NATURAL`, `MMD_ATA`, `MMD_AT_PLUT_A`, or `COLAMD`, which indicates different permutation methods that are built in these methods. The object returned by `splu` and `spilu` accounts for such permutations, and the permutation vectors are available via the `perm_c` and `perm_r` attributes.

In [ ]:
lu.L *lu.U -A

In [ ]:
lu.perm_r, lu.perm_c

In [ ]:
def sp_permute(A, perm_r, perm_c):
    """ permute rows and columns of A """
    M, N = A.shape
    # row permumation matrix
    Pr = sp.coo_matrix((np.ones(M), (perm_r, np.arange(N)))).tocsr()
    # column permutation matrix
    Pc = sp.coo_matrix((np.ones(M), (np.arange(M), perm_c))).tocsr()
    return Pr.T *A *Pc.T

In [ ]:
sp_permute(lu.L *lu.U, lu.perm_r, lu.perm_c) -A

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4))
ax1.spy(lu.L)
ax2.spy(lu.U)
ax3.spy(A);

By default, the direct sparse linear solver in Scipy uses the [`SuperLU`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.SuperLU.html) package. An alternative sparse matrix solver that also can be used in Scipy is the `UMFPACK` package, although this package is not bundled with Scipy and requires that the [`scikit-umfpack`](https://scikit-umfpack.github.io/scikit-umfpack/) Python library is installed. If `scikit-umfpack` is available and the `use_umfpack` argument to the `sp.linalg.spsolve` function is `True`, then the `UMFPACK` is used instead of `SuperLU`. 

Whether `SuperLU` or `UMFPACK` gives better performance varies from problem to problem, so it is worth having both installed and testing both for any given problem

In [ ]:
# use_umfpack=True is only effective if scikit-umfpack is installed
x = sp.linalg.spsolve(A, b, use_umfpack=True); x

The `sp.spsolve` function is an interface to direct solvers, which internally performs matrix
factorization. 

An alternative approach is to use iterative methods that originate in optimization. The Scipy `sparse.linalg` module contains several functions for iterative solution of sparse linear problems: For example, [`bicg`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.bicg.html) (biconjugate gradient method), [`bicgstab`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.bicgstab.html) (biconjugate gradient stabilized method), [`cg`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.cg.html)(conjugate gradient), [`gmres`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.gmres.html) (generalized minimum residual), and [`lgmres`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.lgmres.html) (loose generalized minimum residual method). 

All of these functions (and a few others) can be used to solve the problem $\mathbf{Ax} = \mathbf{b}$ by calling the function with $\mathbf{A}$ and $\mathbf{b}$ as arguments, and they all return a tuple `(x,info)` where `x` is the solution and `info` contains additional information about the solution process (`info=0` indicates success, and it is positive for convergence error, and negative for input error). For example:

In [ ]:
x, info = sp.linalg.bicgstab(A, b); x

In [ ]:
x, info = sp.linalg.cg(A, b); x

In [ ]:
x, info = sp.linalg.lgmres(A, b, atol=1e-5); x

Iterative solver may have an advantage over direct solvers for very large problems,
where direct solvers may require excessive memory usage due to undesirable fill-ins. In contrast, iterative solvers only require to evaluate sparse matrix-vector multiplications, and therefore do not suffer from fill-in problems, but on the other hand, they might have slow convergence for many problems, especially if not properly preconditioned

#### A matrix permutation method: [reverse_cuthil_mckee](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csgraph.reverse_cuthill_mckee.html)

In [ ]:
N = 25
A = sp.diags([1, -2, 1], [8, 0, -8], shape=[N, N], format='csc')
perm = sp.csgraph.reverse_cuthill_mckee(A); perm

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))
ax1.spy(A)
ax2.spy(sp_permute(A, perm, perm));

#### Performance comparison: Sparse/Dense

In [ ]:
# compare performance of solving Ax=b vs system size N,
# where A is the sparse matrix for the 1d poisson problem
import time

def setup(N):
    A = sp.diags([1,-2,1], [1,0,-1], shape=[N, N], format='csr')
    b = -np.ones(N)
    return A, A.todense(), b

reps = 10

N_vec = np.arange(10, 2000, 10)
t_sparse = np.empty(len(N_vec))
t_dense = np.empty(len(N_vec))
for idx, N in enumerate(N_vec):
    A, A_dense, b = setup(N)
    t = time.time()
    for r in range(reps):
        x = np.linalg.solve(A_dense, b)
    t_dense[idx] = (time.time() -t)/reps

    t = time.time()
    for r in range(reps):
        x = sp.linalg.spsolve(A, b, use_umfpack=True)
    t_sparse[idx] = (time.time() -t)/reps

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

ax.plot(N_vec, t_dense *1e3, '.-', label="dense")
ax.plot(N_vec, t_sparse *1e3, '.-', label="sparse")
ax.set_xlabel(r"$N$", fontsize=16)
ax.set_ylabel("Elapsed Time, [ms]", fontsize=16)
ax.legend(loc=0)

ax.set_xlim(0, 2000)
ax.set_ylim(0, 400)

fig.tight_layout()

### Eigenvalue problems

Sparse eigenvalue and singular-value problems can be solved using the [`sp.linalg.eigs`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.eigs.html) and [`sp.linalg.svds`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.svds.html)
functions, respectively. For real symmetric or complex hermitian matrices, the eigenvalues (which in this
case are real) and eigenvectors can also be computed using [`sp.linalg.eigsh`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.eigsh.html). These functions do not compute all eigenvalues or singular values, but rather compute a given number of eigenvalues and
vectors (the default is six).

For example, to compute the largest four eigenvalues for the sparse matrix of the one-dimensional
Poisson problem (of system size 10 x 10),

In [ ]:
N = 10
A = sp.diags([1, -2, 1], [1, 0, -1], shape=[N, N], format='csc')
evals, evecs = sp.linalg.eigs(A, k=4, which='LM')

In [ ]:
evals

In [ ]:
evecs

In [ ]:
np.allclose(A.dot(evecs[:, 0]), evals[0] *evecs[:, 0])

For this particular example, the sparse matrix $\mathbf{A}$ is symmetric, so instead of `sp.linalg.eigs` we could use `sp.linalg.eigsh` instead, and in doing so we obtain an eigenvalue array with real-valued elements:

In [ ]:
evals, evecs = sp.linalg.eigsh(A, k=4, which='LM'); evals

By changing the argument `which='LM'` (for largest magnitude) to `which='SM'` (smallest magnitude), we
obtain a different set of eigenvalues and vector (those with smallest magnitude).

In [ ]:
evals, evecs = sp.linalg.eigs(A, k=4, which='SM'); evals

In [ ]:
def sp_eigs_sorted(A, k=6, which='SR'):
    """ compute and return eigenvalues sorted by real value """
    evals, evecs = sp.linalg.eigs(A, k=k, which=which)
    idx = np.real(evals).argsort()
    return evals[idx], evecs[idx]

In [ ]:
evals, evecs = sp_eigs_sorted(A, k=4, which='SM'); evals

As a less trivial example, consider the spectrum of lowest eigenvalues of the linear combination $(1 -x) M_1 +x M_2$ of random sparse matrices $M_1$ and $M_2$. We can use the [`sp.rand`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.rand.html) function to generate two random sparse matrices, and by repeatedly using `sp_eigs_sorted` to find the smallest 25 eigenvalues of the $(1 -x) M_1 +x M_2$ matrix for different values of $x$,

In [ ]:
x_vec = np.linspace(0, 1, 50)

N = 100
# seed sp.rand with random_state to obtain a reproducible result
M1 = sp.rand(N, N, density=0.2, random_state=112312321)
M2 = sp.rand(N, N, density=0.2, random_state=984592134)

In [ ]:
evals = np.array([sp_eigs_sorted((1 -x) *M1 +x *M2, k=25)[0] for x in x_vec])

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

for idx in range(evals.shape[1]):
    ax.plot(x_vec, np.real(evals[:,idx]), lw=0.5)

ax.set_xlabel(r"$x$", fontsize=16)
ax.set_ylabel(r"eig. vals. of $(1-x)M_1+xM_2$", fontsize=16)
ax.set_xlim(0, 1)
ax.set_ylim(-2.5, -0.5)
ax.tick_params(which='both', direction='in')

fig.tight_layout()

## Graphs and networks

Representing graphs as adjacency matrices is another important application of sparse matrices. In an
adjacency matrix, an element describes which nodes in a graph are connected to each other. Consequently,
if each node is only connected to a small set of other nodes, the adjacency matrix is sparse.
The `csgraph` module in the Scipy `sparse` module provides functions for processing such graphs, including methods for traversing a graph using different methods (breadth-first and depth-first traversals, for example) and for computing shortest paths between nodes in a graph, and so on

For a more comprehensive framework for working with graphs, there is the [`NetworkX`](https://networkx.github.io/documentation/stable/) Python library.
It provides utilities for creating and manipulating undirected and directed graphs, and also implements
many graph algorithms, such as finding minimum paths between nodes in a graph. Here we assume that
the `networkx` library is imported under the name `nx`. Using this library, we can, for example, create an undirected graph by initiating an object of the class `nx.Graph`. Any hashable Python object can be stored as nodes in a `Graph` object, which makes it very flexible data structure.

For example, we can create a simple graph with node data that are integers using [`nx.Graph`](https://networkx.github.io/documentation/networkx-1.10/tutorial/tutorial.html)`()` and
the `add_node` method, or `add_nodes_from` to add multiple nodes in one go. The `nodes` method returns a
list of nodes:

In [ ]:
g = nx.Graph()
g.add_node(1)
g.nodes()

In [ ]:
g.add_nodes_from([3, 4, 5])
g.nodes() 

To connect nodes we can add edges using `add_edge`. We pass the labels of the two nodes we want to
connect as arguments. To add multiple edges, we can use `add_edges_from`, and pass to it a list of tuples of nodes to connect. The `edges` method returns a list of edges:

In [ ]:
g.add_edge(1, 2)
g.edges()

In [ ]:
g.add_edges_from([(3, 4), (5, 6)])
g.edges()

To represent edges between nodes that have weights associated with them (for example, a distance), we
can use `add_weighted_edges_from`, to which we pass a list of tuples that also contains the weight factor for each edge, in addition to the two nodes. When calling the `edges` method, we can additionally give argument `data=True` to indicate that also the edge data should be included in the resulting list

In [ ]:
g.add_weighted_edges_from([(1, 3, 1.5), (3, 5, 2.5)])
g.edges(data=True)

Note that if we add edges between nodes that do not yet exist in the graph, they are seamlessly added.
For example, in the following code, we add a weighted edge between node 6 and 7. Node 7 does not previously exist in the graph, but when adding an edge to it, it is automatically created and added to the graph:

In [ ]:
g.add_weighted_edges_from([(6, 7, 1.5)])
g.nodes()

In [ ]:
g.edges()

With these basic fundamentals in place, we are already prepared to look at a more complicated example
of a graph. In the following we will build a graph from a dataset stored in a JSON file called
`tokyo-metro.json`, which we load using the Python standard library module `json`:

In [ ]:
import json
with open("./files/tokyo-metro.json") as f:
    data = json.load(f)

The result of loading the JSON file is a dictionary `data` that contains metro line descriptions. For each
line, there is a list of travel times between stations (`travel_times`), a list of possible transfer points to other
lines (`transfer`), as well as the line color:

In [ ]:
data.keys()

In [ ]:
data["C"]

The format of the `transfers` list is `[['C3', 'F15'], ...]`, indicating that it is possible
to transfer from C line to F line at station C3 to station F15. 
The format of the `travel_times` list is `[['C1', 'C2', 2], ['C2', 'C3', 2], ...]`, indicating that
it takes two minutes to travel between the stations C1 and C2, and two minutes to travel between
C2 and C3, etc. The `transfers` and `travel_times` are
directly suitable for feeding to `add_weighed_edges_from` and `add_edges_from`, and we can therefore easily
create a graph for representing the metro network by iterating over each metro line dictionary and call
these methods:

In [ ]:
g = nx.Graph()

for line in data.values():
    g.add_edges_from(line["transfers"])    
    g.add_weighted_edges_from(line["travel_times"])

In [ ]:
g.edges(data=True)

The line transfer edges do not have edge weights, so let's first mark all transfer edges by adding a new Boolean attribute `transfer` to each edge:

In [ ]:
for n1, n2 in g.edges():
    g[n1][n2]["transfer"] = "weight" not in g[n1][n2]

In [ ]:
g.edges(data=True)

In [ ]:
g.number_of_nodes()

In [ ]:
list(g.nodes())[:5]

In [ ]:
g.number_of_edges()

In [ ]:
list(g.edges())[:5]

Next, for plotting purposes, we create two lists of edges containing transfer edges and on-train edges:

In [ ]:
on_foot = [edge for edge in g.edges() if g.get_edge_data(*edge)["transfer"]]
on_train = [edge for edge in g.edges() if not g.get_edge_data(*edge)["transfer"]]

and we also create a list with colors corresponding to each node in the network:

In [ ]:
colors = [data[n[0].upper()]["color"] for n in g.nodes()]

To visualize the graph, we can use the Matplotlib-based drawing routines in the `networkx` library:
We use `nx.draw_networkx` to draw each node, `nx.draw_networkx_labels` to draw the labels to the nodes, 
`nx.draw_network_edges` to draw the edges. We call `nx.draw_network_edges` twice, with the edge lists for transfers
(`on_foot`) and on-train (`on_train`) connections, and color the links as blue and black, respectively, using the
`edge_color` argument. The layout of the graph is determined by the `pos` argument to the drawing functions.
Here we used the `graphviz_layout` to layout the nodes. All drawing functions also accept a Matplotlib
axes instance via the `ax` argument.

In [ ]:
# !conda install -c alubbock graphviz pygraphviz
# Please install graphviz and set 'path' environment for Windows 10 (See ./files/graphvis_installation.txt)

from networkx.drawing.nx_agraph import graphviz_layout

In [ ]:
%matplotlib qt

fig, ax = plt.subplots(1, 1, figsize=(20, 15))

pos = graphviz_layout(g, prog="neato")
nx.draw_networkx(g, pos, ax=ax, node_size=450, node_color=colors)
nx.draw_networkx_labels(g, pos=pos, ax=ax, font_size=1)
nx.draw_networkx_edges(g, pos=pos, ax=ax, edgelist=on_train, width=2)
nx.draw_networkx_edges(g, pos=pos, ax=ax, edgelist=on_foot, edge_color="red", width=2)

# removing the default axis on all sides:
for side in ['bottom','right','top','left']:
    ax.spines[side].set_visible(False)

# removing the axis labels and ticks
ax.set_xticks([])
ax.set_yticks([])
ax.xaxis.set_ticks_position('none')
ax.yaxis.set_ticks_position('none')

Once the network has been constructed, we can use the many graph algorithms provided by the
NetworkX library to analyze the network. For example, to compute the degree (that is, the number of
connections to a node) of each node, we can use the `degree` method

In [ ]:
g.degree()

For this graph, the degree of a node can be interpreted as the number of connections to a station. We can
easily search for the most highly connected station in the network by using the `degree` method, the `values` method of the resulting Python dictionary, and the `max` function to find the highest degree in the network

In [ ]:
g_degree = dict(g.degree())
d_max = max(g_degree.values())
[(n, d) for (n, d) in g_degree.items() if d == d_max]

We can also compute the closest path between two points in the network using `nx.shortest_path`. For example, the optimal
traveling route (assuming no waiting time and instantaneous transfer) for traveling between `Y24` and `C19` is:

In [ ]:
p = nx.shortest_path(g, "Y24", "C19"); p

Given a path on this form, we can also directly evaluate the travel time by summing up the weight
attributes of neighboring nodes in the path:

In [ ]:
np.sum([g[p[n]][p[n +1]]["weight"] for n in range(len(p) -1) if "weight" in g[p[n]][p[n +1]]])

The result suggests that it takes 35 minutes to travel from `Y24` to `C19`. Since the transfer nodes do not have a weight associated with them, the train transfers are effectively assumed to be instantaneous. 

It may be reasonable to assume that a train transfer takes about 5 minutes, and to take this into account in the shortest path and travel time computation, we can update the transfer nodes and add a weight of 5 to each of them. To do this we create a copy of the graph using the `copy` method, and iterate through the edges and update those with `transfer` attribute set to `True`:

In [ ]:
h = g.copy()
for n1, n2 in h.edges():
    if h[n1][n2]["transfer"]:
        h[n1][n2]["weight"] = 5
p = nx.shortest_path(h, "Y24", "C19")

In [ ]:
p

In [ ]:
np.sum([h[p[n]][p[n +1]]["weight"] for n in range(len(p) -1)])

With this method, we can of course compute the optimal path and travel time between arbitrary nodes
in the network. As another example, we also compute the shortest path and traveling time between Z1 and
H16 (32 minutes):

In [ ]:
p = nx.shortest_path(h, "Z1", "H16")
np.sum([h[p[n]][p[n +1]]["weight"] for n in range(len(p) -1)])

The NetworkX representation of a graph can be converted to an adjacency matrix in the form of a Scipy
sparse matrix using the `nx.to_scipy_sparse_matrix`, after which we can also analyze the graph with the
routines in the `sp.csgraph` module

In [ ]:
A = nx.to_scipy_sparse_matrix(g); A

As an example of this, we convert the Tokyo Metro graph to an adjacency
matrix and compute its reverse Cuthill-McKee ordering (using `sp.csgraph.reverse_cuthill_mckee`, which is a reordering that reduces the maximum distance of the matrix elements from the diagonal), and permute the matrix with this ordering

In [ ]:
perm = sp.csgraph.reverse_cuthill_mckee(A)


%matplotlib inline
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))

ax1.spy(A, markersize=2)
ax2.spy(sp_permute(A, perm, perm), markersize=2)

fig.tight_layout()

## Versions

In [ ]:
import pygraphviz
print("numpy: ", np.__version__)
print("scipy: ", scipy.__version__)
print("matplotlib: ", mpl.__version__)
print("networkx: ", nx.__version__)
print("pygraphviz: ", pygraphviz.__version__)